In [77]:
pip install pymysql


Defaulting to user installation because normal site-packages is not writeable
  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.


In [79]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pyodbc
import pymysql

In [95]:


# Define connection parameters
host = "localhost"
user = "root"
password = "8127"  # Your password
database = "movie_db"  # Replace with your actual database name

# Establish connection
conn = pymysql.connect(host=host, user=user, password=password)

# Create a cursor object
cursor = conn.cursor()

# Create database if not exists
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
print("✅ Database Checked/Created!")

# Switch to movie database
conn.select_db(database)

# Create movies table (Modify columns as needed)
cursor.execute("""
CREATE TABLE IF NOT EXISTS movies (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Movie_name VARCHAR(255),
    genre VARCHAR(100),
    release_year INT,
    rating FLOAT
)
""")

print("✅ Movies Table Checked/Created!")

# Close connection
cursor.close()
conn.close()



FileNotFoundError: [Errno 2] No such file or directory: 'scraped_movies.csv'

In [33]:
url = "https://www.imdb.com/list/ls002939221/?genres=drama"
driver  = webdriver.Chrome()
driver.get(url)
driver.maximize_window()

In [49]:
# For scraping movies name
name = driver.find_element(By.CLASS_NAME,"ipc-title__text").text
name

'1. American Psycho'

In [51]:
#for rating
rating = driver.find_element(By.CLASS_NAME, "ipc-rating-star--rating").text
rating

'7.6'

In [53]:
# For duration
year = driver.find_element(By.CLASS_NAME, "sc-d5ea4b9d-7").text
year

'2000'

In [55]:
# For year
duration = driver.find_element(By.CLASS_NAME, "sc-d5ea4b9d-6").text
duration.split("\n")[1]


'1h 42m'

In [57]:
driver.find_element(By.CLASS_NAME, "sc-b0901df4-0").text

'64'

In [59]:

genres = ["drama","thriller","action","comedy","adventure","romance","sci-fi","mystery","horror","crime"]


In [61]:
# director
driver.find_element(By.CLASS_NAME, "sc-d49a611d-2").text

'Mary Harron'

In [63]:
driver.find_element(By.CLASS_NAME,"sc-d49a611d-3").text

'Director'

In [35]:
movie = []
for genre in genres:
    url = f"https://www.imdb.com/list/ls002939221/?genres={genre}"
    driver.get(url)
    
    # Scroll to the bottom to ensure all elements are loaded
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # Allow time for the page to load fully
    
    # Find all movies in the list
    movie_elements = driver.find_elements(By.CLASS_NAME, "sc-e2db8066-1")
    time.sleep(1)
    
    for movie_element in movie_elements:
        try:
            # Extract movie details with error handling
            name = movie_element.find_element(By.CLASS_NAME, "ipc-title__text").text
            
            # Handle cases where rating might not exist
            try:
                rating = movie_element.find_element(By.CLASS_NAME, "ipc-rating-star--rating").text
            except:
                rating = "N/A"  # Default value if rating is not found
            try:
                director = movie_element.find_element(By.CLASS_NAME, "sc-d49a611d-2").text
            except:
                director = "N/A"

            # Handle cases where duration and year are missing
            try:
                details = movie_element.find_element(By.CLASS_NAME, "sc-d5ea4b9d-6").text.split("\n")
                year = details[0] if len(details) > 0 else "N/A"
                duration = details[1] if len(details) > 1 else "N/A"
            except:
                year = "N/A"
                duration = "N/A"

            # Append the extracted data
            data = {
                "Movie_name": name,
                "Rating": rating,
                "Duration": duration,
                "Release_year": year,
                "Genre": genre,
                "Director" : director
            }
            movie.append(data)
        
        except Exception as e:
            # Log the error for debugging
            print(f"Error scraping movie details: {e}")
            continue



In [37]:
scrape = driver.find_elements(By.CLASS_NAME, "sc-e2db8066-1")
scrape

[<selenium.webdriver.remote.webelement.WebElement (session="e46454a5db61bbd411944f51a261820c", element="f.2E4F55CF2765AC91AA8AD795D2C0856C.d.82B889DAFFB16AE26A9A728A3272B36C.e.92811")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e46454a5db61bbd411944f51a261820c", element="f.2E4F55CF2765AC91AA8AD795D2C0856C.d.82B889DAFFB16AE26A9A728A3272B36C.e.92812")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e46454a5db61bbd411944f51a261820c", element="f.2E4F55CF2765AC91AA8AD795D2C0856C.d.82B889DAFFB16AE26A9A728A3272B36C.e.92813")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e46454a5db61bbd411944f51a261820c", element="f.2E4F55CF2765AC91AA8AD795D2C0856C.d.82B889DAFFB16AE26A9A728A3272B36C.e.92814")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e46454a5db61bbd411944f51a261820c", element="f.2E4F55CF2765AC91AA8AD795D2C0856C.d.82B889DAFFB16AE26A9A728A3272B36C.e.92815")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e46454a5db

In [39]:
movie

[{'Movie_name': '1. American Psycho',
  'Rating': '7.6',
  'Duration': '1h 42m',
  'Release_year': '2000',
  'Genre': 'drama',
  'Director': 'Mary Harron'},
 {'Movie_name': '2. Battle Royale',
  'Rating': '7.5',
  'Duration': '1h 54m',
  'Release_year': '2000',
  'Genre': 'drama',
  'Director': 'Kinji Fukasaku'},
 {'Movie_name': '3. Crouching Tiger, Hidden Dragon',
  'Rating': '7.9',
  'Duration': '2h',
  'Release_year': '2000',
  'Genre': 'drama',
  'Director': 'Ang Lee'},
 {'Movie_name': '5. Frequency',
  'Rating': '7.4',
  'Duration': '1h 58m',
  'Release_year': '2000',
  'Genre': 'drama',
  'Director': 'Gregory Hoblit'},
 {'Movie_name': '6. Ginger Snaps',
  'Rating': '6.8',
  'Duration': '1h 48m',
  'Release_year': '2000',
  'Genre': 'drama',
  'Director': 'John Fawcett'},
 {'Movie_name': '7. Gladiator',
  'Rating': '8.5',
  'Duration': '2h 35m',
  'Release_year': '2000',
  'Genre': 'drama',
  'Director': 'Ridley Scott'},
 {'Movie_name': '10. Malena',
  'Rating': '7.4',
  'Duration

In [41]:
df = pd.DataFrame(movie)

In [43]:
df

,Movie_name,Rating,Duration,Release_year,Genre,Director
0,1. American Psycho,7.6,1h 42m,2000,drama,Mary Harron
1,2. Battle Royale,7.5,1h 54m,2000,drama,Kinji Fukasaku
2,"3. Crouching Tiger, Hidden Dragon",7.9,2h,2000,drama,Ang Lee
3,5. Frequency,7.4,1h 58m,2000,drama,Gregory Hoblit
4,6. Ginger Snaps,6.8,1h 48m,2000,drama,John Fawcett
...,...,...,...,...,...,...
2002,832. Extraction II,7.0,2h 2m,2023,crime,Sam Hargrave
2003,843. House of Sand and Fog,7.5,2h 6m,2003,crime,Vadim Perelman
2004,847. Hit Man,6.8,1h 55m,2023,crime,Richard Linklater
2005,848. Shakhahaari,7.5,2h 25m,2024,crime,Sandeep Sunkad


In [45]:
df.to_csv("Movies.csv")

In [47]:
df['Rating'].unique()

array(['7.6', '7.5', '7.9', '7.4', '6.8', '8.5', '7.2', '7.0', '6.9',
       '7.3', '8.0', '7.8', '8.1', '6.0', '6.4', '8.9', '6.7', '8.3',
       '6.6', '8.8', '7.1', '8.2', '7.7', '6.3', '6.5', '9.0', '6.2',
       '5.3', '8.4', '6.1', '5.8', '5.5', '4.9', '5.7', '5.9', '5.4',
       '5.2', '5.6', '5.1', '4.8', '5.0', '8.7'], dtype=object)

In [97]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="8127",
    database="movies_db"
)

cursor = conn.cursor()

# Load CSV file
df = pd.read_csv("Movies.csv")  # Replace with your file name

# Rename columns to match SQL table
df.rename(columns={"Movie_name": "movie_name", "Rating": "rating", "Duration": "duration",
                   "Release_year": "release_year", "Genre": "genre", "Director": "director"}, inplace=True)

# Clean Data
df["release_year"] = pd.to_numeric(df["release_year"], errors="coerce")
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df = df.dropna()  # Remove rows with missing values

# Insert Data into MySQL
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO movies (movie_name, rating, duration, release_year, genre, director)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row["movie_name"], row["rating"], row["duration"], row["release_year"], row["genre"], row["director"]))

# Commit and Close
conn.commit()
cursor.close()
conn.close()

print("✅ Scraped Data Successfully Loaded into MySQL!")

ProgrammingError: (1146, "Table 'movies_db.movies' doesn't exist")